In [3]:
import requests
import csv
from datetime import datetime, timedelta

api_key = ''

# Endpoint for the NewsAPI 'everything' request
url = 'https://newsapi.org/v2/everything'

# Calculate dates for the past year
today = datetime.now()
from_date = (today - timedelta(days=30)).strftime('%Y-%m-%d')
to_date = today.strftime('%Y-%m-%d')

# Parameters for the API request
params = {
    'apiKey': api_key,
    'sources': 'cbc-news',  
    'q': 'markets OR business OR finance', 
    'from': from_date,
    'to': to_date,
    'pageSize': 100,  
    'language': 'en',
    'sortBy': 'publishedAt',
}

all_articles = []
page = 1

while True:
    params['page'] = page
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        articles = data['articles']
        total_results = data['totalResults']

        if not articles:
            break

        all_articles.extend(articles)

        if len(all_articles) >= 100 or len(all_articles) >= total_results:
            break

        page += 1
    else:
        print('Error:', response.status_code)
        print(response.text)
        break
all_articles = all_articles[:100]


with open('cbc_articles.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Title', 'URL', 'Description', 'Content']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for article in all_articles:
        writer.writerow({
            'Title': article.get('title', ''),
            'URL': article.get('url', ''),
            'Description': article.get('description', ''),
            'Content': article.get('content', '')
        })

print(f"Successfully wrote {len(all_articles)} articles to cbc_articles.csv")


Successfully wrote 100 articles to cbc_articles.csv
